In [1]:
from nlpsandboxclient import client
from copy import deepcopy

In [38]:
DATA_HOST = "http://localhost:8081/api/v1"
ANNO_HOST = "http://localhost:8080/api/v1"
ANNOTATION_STORE = 'neuroner'
DATASET_NAME = 'mayo-test-dataset'


In [ ]:
datasets = client.list_datasets(host=DATA_HOST)

for dataset in datasets:
      print(dataset)

In [ ]:
example_note = {
   "identifier": "note-3",
   "type": "loinc:LP29684-5",
   "patientId": "507f1f77bcf86cd799439111",
   "text": "On 12/26/2020, Jake Li Kg, Mr. Rof, david@uw.edu, sda@gmail.com, Ms. Chloe Price met with 232323 Dr. Prescott id is 32243 in Seattle, WA, 206-434-3545, 343-2423343, 754-33-2343, jake@email.com, l232,1232 , Thai, wong, Linda, at Seattle, WA, hospital."
}

annotations = client.annotate_note(host=ANNO_HOST,
                            note=example_note,
                            tool_type="nlpsandbox:contact-annotator")

print(annotations)



In [22]:
types = ["date","location","id","contact","person-name"]
type_dict = {"date":"textDateAnnotations", 
             "person-name":"textPersonNameAnnotations",
             "location":"textLocationAnnotations",
             "id":"textIdAnnotations",
             "contact":"textContactAnnotations"}

# Option 1

In [ ]:
# option1 : annotate only 1 PHI category. In this example, date is annotated
type = 'location'
notes = client.list_notes(host=DATA_HOST,
                  dataset_id=DATASET_NAME,
                  fhir_store_id="evaluation")

test_notes = list(notes)
print(len(test_notes))

annotation_list = []
converted_annotation_list = []
for i in test_notes:
    note_name= i.pop('note_name', None)
    note_id = note_name[-6:]
    print(note_id)
    # There is an error in annotation generated from  client.annotate_note, the note_name is "dataset/.."  but it should be "datasets/.."
    note_name = "datasets"+note_name[7:]
    converted_annotation = {"annotationSource":{"resourceSource":{"name":note_name}},"textContactAnnotations": [], "textCovidSymptomAnnotations":[],"textDateAnnotations":[],"textIdAnnotations":[],"textLocationAnnotations": [],"textPersonNameAnnotations": []}
    import time
    time.sleep(1)
    
    
    annotations = client.annotate_note(host=ANNO_HOST,
                            note=i,
                            tool_type=f"nlpsandbox:{type}-annotator")
    # original idType is other but it's not accepted, a quick fix to change 'other' 

    #for annotation in annotations["textIdAnnotations"]:
        #annotation['idType'] = 'account'
    #print(annotations[type_dict[type]])
    converted_annotation[type_dict[type]] = annotations[type_dict[type]]
    converted_annotation_list.append(converted_annotation)
    

In [ ]:
len(converted_annotation_list)


In [ ]:
client.store_annotations(host=DATA_HOST,
                 dataset_id=DATASET_NAME,
                 annotation_store_id=ANNOTATION_STORE,
                 annotations=converted_annotation_list,
                 delete_existing_annotations = True)


In [ ]:
client.evaluate(host=DATA_HOST,
               dataset_id=DATASET_NAME,
               annotation_store_id=ANNOTATION_STORE,
               tool_type="nlpsandbox:contact-annotator")

In [33]:
####cross-check the prediction and the goldstandard#####

annos = client.list_annotations(host=DATA_HOST,
                 dataset_id=DATASET_NAME,
                 annotation_store_id="goldstandard")
anno_list=list(annos)


In [81]:

note_ob = client.list_notes(host=DATA_HOST,
                  dataset_id=DATASET_NAME,
                  fhir_store_id="evaluation")
notes = list(note_ob)
def get_note(notes,id):
    return notes[RANDOM_NOTE_ID]["text"]


In [ ]:
format_map = {"date":"dateFormat","location":"locationType","id":"idType","contact":"contactType"}
#types = ["date","location","id","contact","person-name"]
types = ["id"]
for type in types:
    with open(f"mismatch_fp_{type}.txt","w") as f:
        for RANDOM_NOTE_ID in range(0, len(anno_list)):
            gold = deepcopy(anno_list)[RANDOM_NOTE_ID][type_dict[type]]
            pred = deepcopy(converted_annotation_list)[RANDOM_NOTE_ID][type_dict[type]]

            print(f"note_id: {deepcopy(converted_annotation_list)[RANDOM_NOTE_ID]['annotationSource']['resourceSource']['name']}")
            for i in pred:
                if type in format_map.keys():
                    i.pop(format_map[type],"confidence")
                    i.pop("confidence")
            for i in gold:
                if type in format_map.keys():
                    i.pop(format_map[type])
                    i.pop("confidence")
            print("-----gold------")
            print(gold)
            print("-----pred------")
            print(pred)
            print("---------------")
 
            for i in pred:
                if i not in gold:
                    f.write(str(i))
                    f.write(str(get_note(notes,RANDOM_NOTE_ID)[i["start"]-20:i["start"]+i["length"]+20]))
                    f.write("\n")
                    print("**********")
                    print(i)
                    print("**********")
                    #print("original ----")
                    #print(get_note(notes,RANDOM_NOTE_ID)[i["start"]:i["start"]+i["length"]])
                    #print("\n")
            '''
            for i in gold:
                if i not in pred:
                    f.write(str(i))
                    
                    print(i)
                    print("original ----")
                    #f.write(str(get_note(notes,RANDOM_NOTE_ID)[i["start"]-20:i["start"]+i["length"]+20]))
                    print(get_note(notes,RANDOM_NOTE_ID)[i["start"]:i["start"]+i["length"]])
                    print("\n")
                    f.write("\n")
            '''  
    #print(gold)
    #print("\n")
    #print(pred)
f.close()
########################################################

# Option 2

In [41]:
# option 2: annotate all 5 PHI categories.
#types = ["person-name","date","location","id","contact"]
types = ["person-name","date"]
notes = client.list_notes(host=DATA_HOST,
              dataset_id=DATASET_NAME,
              fhir_store_id="evaluation")
test_notes = list(notes)
annotation_list = []
converted_annotation_list = []
for i in test_notes:
    note_name= i.pop('note_name', None)
    note_id = note_name[-6:]
    # There is an error in annotation generated from  client.annotate_note, the note_name is "dataset/.."  but it should be "datasets/.."
    note_name = "datasets"+note_name[7:]
    converted_annotation = {"annotationSource":{"resourceSource":{"name":note_name}},"textContactAnnotations": [], "textCovidSymptomAnnotations":[],"textDateAnnotations":[],"textIdAnnotations":[],"textLocationAnnotations": [],"textPersonNameAnnotations": []}
    for type in types:
        annotations = client.annotate_note(host=ANNO_HOST,
                                note=i,
                                tool_type=f"nlpsandbox:{type}-annotator")
        converted_annotation[type_dict[type]] = annotations[type_dict[type]]
    converted_annotation_list.append(converted_annotation)

In [ ]:
client.store_annotations(host=DATA_HOST,
                 dataset_id=DATASET_NAME,
                 annotation_store_id=ANNOTATION_STORE,
                 annotations=converted_annotation_list,
                 delete_existing_annotations = True)

In [ ]:
print(f"dataset: {DATASET_NAME}")
print(f"annotation store: {ANNOTATION_STORE}")

In [ ]:
for type in types:
    print("*"*40 + type + "*"*40)
    client.evaluate(host=DATA_HOST,
                                   dataset_id=DATASET_NAME,
                                   annotation_store_id=ANNOTATION_STORE,
                                   tool_type=f"nlpsandbox:{type}-annotator")

In [18]:
# list annotation
gold = client.list_annotations(host=DATA_HOST,
                  dataset_id=DATASET_NAME,
                  annotation_store_id="goldstandard")
gold = list(gold)